In [ ]:
import src.pmcmc.main as pmcmc_main
import pandas as pd
import os
from src.utils import paths


loc_csv = os.path.join(paths.DATASETS_DIR, 'locations.csv')
loc_df = pd.read_csv(loc_csv)

In [2]:
loc_codes = loc_df['location']

In [6]:
print(loc_codes.values)

['US' '01' '02' '04' '05' '06' '08' '09' '10' '11' '12' '13' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '44' '45'
 '46' '47' '48' '49' '50' '51' '53' '54' '55' '56' '72']


In [8]:
for code in loc_codes.values:
    print("'", code, "',", sep="", end="")

'US','01','02','04','05','06','08','09','10','11','12','13','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56','72',

In [9]:
from src.utils.location_codes import location_codes

In [11]:
location_codes[0]

'01'

In [16]:
!python -m src.scripts.all_locations_beta_test

Running Particle Filter:   1%|▏                 | 3/308 [00:16<26:43,  5.26s/it]^C
                                                                                Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/andrew/PycharmProjects/pmcmc_forecast/src/scripts/all_locations_beta_test.py", line 14, in <module>
    pool.map(run_pmcmc, location_codes)
  File "/usr/lib64/python3.12/multiprocessing/pool.py", line 367, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/multiprocessing/pool.py", line 768, in get
    self.wait(timeout)
  File "/usr/lib64/python3.12/multiprocessing/pool.py", line 765, in wait
    self._event.wait(timeout)
  File "/usr/lib64/python3.1

In [1]:
import os
import numpy as np 
import pandas as pd
from src.utils import paths

locations = ['06', '08', '36', '47']
target_dates = pd.read_csv('../datasets/target_dates.csv')

In [3]:
dates = target_dates['date'].values
dates

array(['2023-10-14', '2023-10-21', '2023-10-28', '2023-11-04',
       '2023-11-11', '2023-11-18', '2023-11-25', '2023-12-02',
       '2023-12-09', '2023-12-16', '2023-12-23', '2023-12-30',
       '2024-01-06', '2024-01-13', '2024-01-20', '2024-01-27',
       '2024-02-03', '2024-02-10', '2024-02-17', '2024-02-24',
       '2024-03-02', '2024-03-09', '2024-03-16', '2024-03-23',
       '2024-03-30', '2024-04-06', '2024-04-13', '2024-04-20',
       '2024-04-27'], dtype=object)

In [5]:

for loc in locations:
    print("Working location ", loc)
    base_dir = os.path.join(paths.OUTPUT_DIR, 'pmcmc_run_20241013')
    loc_dir = os.path.join(base_dir, loc)
    beta_file_path = os.path.join(loc_dir, 'mle_betas.csv')
    df = pd.read_csv(beta_file_path)
 
    # Average over the particles
    row_means = df.mean(axis=0)
    # Transpose the DataFrame
    transposed_df = row_means.to_frame() 
    num_rows = len(transposed_df)
    end_date = pd.to_datetime("2024-04-27")
    date_range = pd.date_range(end=end_date, periods=num_rows)
    transposed_df["date"] = date_range
    transposed_df.rename({0: "beta"}, axis="columns", inplace=True)
    
    df = transposed_df
    for target_date in target_dates["date"]:
        start_date = pd.to_datetime(target_date) - pd.Timedelta(days=50)
        filtered_df = df[(df["date"] > start_date) & (df["date"] <= target_date)]
        filtered_df.insert(0, "time_0", range(len(filtered_df)))
        filtered_df = filtered_df.drop(columns=["date"])
        file_path = os.path.join(paths.PF_OUTPUT_DIR, target_date, f"{loc}.csv")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        filtered_df.to_csv(file_path, index=False)

Working location  06
Working location  08
Working location  36
Working location  47
